# Regularizers as memory

The greatest challenge in modern AI research is the limiting returns computational work. 
While the greatest advances can only be afforded by large technology firms, even they 
cannot afford to push results further. A clear plateau has developed. 
Insufficient computational efficiency motivates a return to theory, begs a question: 
_where is waste most significant?_ 

The advent of catestrophic forgetting shows raw, non-compressed information must be continually re-applied 
if it is not to be forgotten. If important information could be preserved--even partially--we would expect 
more efficient computation. In short, _this work targets the realization of memory_.

Regularized likelihood equations have a Lagrangian form, so implicitly describe geometric constraints on estimates.
For example, here's an estimate constrained to an L2-sphere in $\Theta$-space.

$$ \hat \theta_{L_2} = \text{arg max}_{\theta \in \Theta} n^{-1} \sum_{i=1}^n \log f_X(X_i;\theta) - \lambda \| \theta \|_2^2 $$

In this work, we'll generalize the regularizer $\| \cdot \|_2^2$ to support alternative geometries, 
in attempting to construct numerically convenient memory approximations. 
Particulary, we'll seek to approximate the following equation.
Note that it introduces quadratic geometric constraints on the estimate. 

$$ \hat \theta = \text{arg max}_{\theta \in \Theta} n^{-1} \sum_{i=1}^n \log f_X(X_i;\theta) - \frac{\lambda}{2} (\theta - \theta_0)^T \mathcal{I}_{\theta_0} (\theta - \theta_0) $$

## RL-reweighted (RLR) estimates 

This first estimate is designed to improve efficiency of sampling in reinforcement learning (RL) by 
up-weighting more-important observations. We'll use this estimate:

$$ \hat \theta_{RLR} = \hat \theta_{n_1+n_2} = \text{arg max}_\theta \left( \sum_{i=1}^{n_1+n_2} \| \hat r_i \|_2^{2b} \right)^{-1} \left( \sum_{i=n_1+1}^{n_1+n_2} \| \hat r_i \|_2^{2b} \log f_X(X_i; \theta) - \sum_{i=1}^{n_1} \frac{\lambda}{2} (\theta - \hat \theta_{n_1+n_2-1})^T H_i (\theta - \hat \theta_{n_1+n_2-1}) \right) $$

where 
- $n_1$ is the number of priviously observed sameples, 
- $n_2$ is the number of newly observed samples, 
- $\hat r_i$ is the estimated reward for observation $i$, 
- $b > 0$ is a bias term,  
- $H_i = \| \hat r_i \|^{2b}_2 \left( \nabla_\theta \log f_X(X_j; \theta)|_{\theta = \hat \theta_{j-1}} \right) \left( \nabla_\theta \log f_X(X_j; \theta)|_{\theta = \hat \theta_{j-1}} \right)^T$ is a Hessian approximation, and
- $\hat \theta_0$ is an initial estimate guess, commonly required in optimization routines. 

Define $ M:= \sum_{i=1}^{n_1}(\theta - \theta_{n_1+n_2-1})^TH_i(\theta - \theta_{n_1+n_2-1})$, which we'll refer to as the "memory term". 
The key heuristic is that $M$, a quadratic regularizer, approximates large-sample likelihoods under maximization. 
This insight can be seen through a Taylor approximation near true $\theta$, designated $\theta_T$.

$\hat \theta_{MLE} = \text{arg max}_\theta n^{-1} \sum_{i=1}^n \log f_X(X_i;\theta) $

$ \approx \text{arg max}_\theta n^{-1}\sum_i \log f_X(X_i; \theta_T) + n^{-1}\sum_i (\theta - \theta_T)^T \nabla_\theta \log f_X(X_i; \theta_T) + n^{-1}2^{-1} \sum_i (\theta - \theta_T)^T (\nabla_\theta^2 \log f_X(X_i; \theta_T))(\theta - \theta_T) $

$ \approx \text{arg max}_\theta n^{-1}\sum_i \log f_X(X_i; \theta_T) + 0 + n^{-1}2^{-1} \sum_i (\theta - \theta_T)^T (\nabla_\theta^2 \log f_X(X_i; \theta_T))(\theta - \theta_T) $

$ \approx \text{arg max}_\theta n^{-1}2^{-1} \sum_i (\theta - \theta_T)^T (\nabla_\theta^2 \log f_X(X_i; \theta_T))(\theta - \theta_T) $

This work basically tests the quality of our $M \approx \sum_i (\theta - \theta_T)^T (\nabla_\theta)^2 \log f_X(X_i; \theta_T)(\theta - \theta_T) $ heuristic.
If true, then we should see that the quadratic regularizer, $M$, affects memory through geometric constraints on the estimator.

Later, we'll experiment with recency bias and rank reductions for $M$.

Notice that $\hat \theta_{RLR}$ is asymptotically equivalent to a redistributed sampling, 
with more observations when they are more important. 
Hence, the essence of this computational efficiency strategy is to 
- add frequency bias to important observations, 
- and preserve their information via geometric estimator constraints.

## $M$ and almost-stationary processes

$M$ is an asymptotic result, so only applies when observations are sufficiently many and produced by a stationary or i.i.d process.
While most applied asymptotic theory assumes $\theta_T$ is static, it could also be possible to work with a $\theta_{T_n}$ on a continuous path in $\Theta$ over $n$.
Let us define an _almost-stationary process_ $X_n$ as satisfying the following.

$$ \forall \tau \in \mathbb{R}, \left(t_1, t_2, \ldots, t_m\right) \in \mathbb{R}_{\geq 0}^m, \mathbb{P}\left[ X_{t_{1+\tau}}, X_{t_{2+\tau}}, \ldots, X_{t_{m+\tau}} ; \theta_{t_1+\tau} \right] \text{is continuous in } \tau$$

The question of whether $\theta_{T_n}$ moves "too quickly" is dependent on an given process. 
If, for $n$ large, $M$ still approximates $\sum_i (\theta - \theta_{T_n})^T (\nabla_\theta^2 \log f_X(X_i; \theta_{T_n}))(\theta - \theta_{T_n})$, 
then we'll say $\theta_{T_n}$ is "sufficiently slow". Here are some examples relating to **transfer learning**.
- If we withheld digits 1 & 2 from MNIST for the first 10000 observations, then started including them afterward, 
we should expect $\theta_{T_n}$ to move too quickly. 
- As reinforcement learning (RL) agent explores its world, it's sampling process indeed deforms, albeit slowly. 
This context should have good opportunities for sufficiently slow changes in $\theta_{T_n}$. 
So, our experiments will focus on RL applications.

Developing transfer learning mechanisms for RL is particularly valuable, 
since it is frequently imagined to be a path to artificial general intelligence (AGI).

Side node: almost-stationary process theory clearly needs development. 
My experience has taught me to test the value of theoretical hypotheses before developing them, 
since theoretical development is far more expensive than computational experimentation.
I am confident in this hypothesis, because my previous theoretical developments are very similar. 
Any new proofs would not be very novel, and would be mere adaptations of prior work. 

## Experiment 1: Cartpole

We start with a _very_ simple example, proving-out concepts.

In [1]:
## Define model 
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random 
import gym 
from tqdm import tqdm 
import numpy as np 
import matplotlib.pyplot as plt 
import copy 

INPUT_DIM = 4
N_ACTIONS = 2
MAX_SAMPLE = 100000
DISCOUNT = .95 
EPS = 1e-5
EXPLORE_PROBABILITY_FUNC = lambda idx: 0.99**idx 
BATCH_SIZE = 30 
LEARNING_RATE = 0.001 
GRAD_CLIP = 10.0 
SHORT_TERM_MEMORY_LENGTH = 3 
LBFGS = False 
ENV_NAME = 'CartPole-v1' 

class Model(nn.Module): 
    def __init__(self, 
            input_dim=INPUT_DIM, 
            n_actions=N_ACTIONS, 
            max_sample=MAX_SAMPLE, 
            discount = DISCOUNT, 
            eps=EPS, 
            explore_probability_func=EXPLORE_PROBABILITY_FUNC, 
            batch_size=BATCH_SIZE, 
            learning_rate=LEARNING_RATE, 
            grad_clip=GRAD_CLIP, 
            short_term_memory_length=SHORT_TERM_MEMORY_LENGTH, 
            lbfgs=LBFGS, 
            env_name=ENV_NAME,
            hessian_sum=None, 
            hessian_denominator=None): 
        super(Model, self).__init__() 
        ## store config 
        self.input_dim = input_dim 
        self.n_actions = n_actions 
        self.max_sample = max_sample 
        self.discount = discount 
        self.eps = eps 
        self.explore_probability_func = explore_probability_func 
        self.batch_size = batch_size 
        self.learning_rate = learning_rate 
        self.grad_clip = grad_clip 
        self.short_term_memory_length = short_term_memory_length 
        self.lbfgs = lbfgs 
        self.env_name = env_name 
        self.hessian_sum = hessian_sum 
        self.hessian_denominator = hessian_denominator
        ## init feed forward net 
        self.fc1 = nn.Linear(input_dim * short_term_memory_length, 32) 
        self.fc1_bn = nn.BatchNorm1d(32) 
        self.fc2 = nn.Linear(32, 32) 
        self.fc2_bn = nn.BatchNorm1d(32) 
        self.fc3 = nn.Linear(32, n_actions) 
        ## init data structures 
        self.observations = [] 
        self.env = None 
        if self.lbfgs: 
            ## Misbehavior observed with large `history_size`, ie. >20 
            ## RAM req = O(history_size * model_dim) 
            self.optimizer = optim.LBFGS(self.parameters(), history_size=5) 
        else: 
            ## LBFGS was giving nan parameters 
            self.optimizer = optim.Adam(self.parameters(), lr=self.learning_rate) 
            pass 
        pass 
    
    def copy(self): 
        out = Model(input_dim=self.input_dim, 
                n_actions=self.n_actions, 
                max_sample=self.max_sample, 
                discount=self.discount, 
                eps=self.eps, 
                explore_probability_func=self.explore_probability_func, 
                batch_size=self.batch_size, 
                learning_rate=self.learning_rate, 
                grad_clip=self.grad_clip, 
                short_term_memory_length=self.short_term_memory_length, 
                lbfgs=self.lbfgs) 
        out.load_state_dict(self.state_dict()) 
        return out 

    def forward(self, x): 
        x = x.reshape([-1, self.input_dim * self.short_term_memory_length]) 
        x = self.fc1(x)
        x = self.fc1_bn(x) 
        x = torch.relu(x) 
        x = self.fc2(x) 
        x = self.fc2_bn(x)  
        x = torch.relu(x) 
        x = self.fc3(x) 
        x = x*x 
        return x 
    
    def get_action(self, env_state): 
        env_state = torch.tensor(env_state).float() 
        env_state = env_state.reshape([1, -1]) 
        predicted_reward_per_action_idx = self.forward(env_state) 
        return int(predicted_reward_per_action_idx.argmax()) 
    
    def store_observation(self, observation): 
        if len(observation) > self.max_sample: 
            observation = observation[1:] 
        self.observations.append(observation) 
        pass 
    
    def clear_observations(self): 
        self.observations = [] 
        pass 

    def convert_observations_to_memory(self): 
        ## convert current observations to a Hessian matrix 
        for obs in self.observations: 
            ## update hessian  
            predicted, target, _ = model._Model__memory_replay(target_model=model, batch_size=None, fit=False, batch=[obs]) 
            loss = F.smooth_l1_loss(predicted, target) 
            loss.backward() 
            grad_vec = torch.cat([p.grad.reshape([-1]) for p in model.parameters()]) 
            outter_product = grad_vec.reshape([-1,1]).matmul(grad_vec.reshape([1,-1])) 
            if self.hessian_sum is None or self.hessian_denominator is None: 
                self.hessian_sum = outter_product 
                self.hessian_denominator = 1 
            else: 
                self.hessian_sum += outter_product 
                self.hessian_denominator += 1 
                pass 
        ## wipe observations, and use memory going forward instead 
        self.clear_observations() 
        pass 
    
    def __memory_replay(self, target_model, batch_size=None, fit=True, batch=None): 
        ## random sample 
        obs = self.observations 
        if batch_size is not None: 
            if batch_size < len(self.observations): 
                obs = random.sample(self.observations, batch_size) 
        if batch is not None: 
            obs = batch 
        ## unpack samples 
        samples = [(torch.tensor(env_state).float(), \
                torch.tensor(reward).float(), \
                torch.tensor(done).int(), \
                torch.tensor(prev_env_state).float(), \
                torch.tensor(action).int()) for \
                (env_state, reward, done, info, prev_env_state, action) in obs] 
        ## build matrices 
        env_state = torch.stack([obs[0] for obs in samples], dim=0) ## inserts dim 0 
        observed_rewards = torch.stack([obs[1] for obs in samples], dim=0) 
        done = torch.stack([obs[2] for obs in samples], dim=0) 
        prev_env_state = torch.stack([obs[3] for obs in samples], dim=0) 
        action = torch.stack([obs[4] for obs in samples], dim=0).reshape([-1, 1]).type(torch.int64) 
        ## calculate target 
        with torch.no_grad(): 
            target_model.eval() 
            predicted_rewards = target_model.forward(env_state) 
            predicted_rewards = torch.max(predicted_rewards, dim=1, keepdim=True).values.reshape([-1]) 
            target = observed_rewards + (1 - done) * self.discount * predicted_rewards 
            target = target.reshape([-1, 1]).detach() 
            pass 
        ## calculate prediction 
        self.zero_grad() 
        if fit: 
            self.train() 
        else:
            self.eval() 
        predicted_rewards = self.forward(prev_env_state) 
        prediction = predicted_rewards.gather(1, action) 
        ## calculate memory regularizer 
        regularizer = None 
        if self.hessian_sum is not None and self.hessian_denominator is not None: 
            t = self.get_parameter_vector() 
            t0 = target_model.get_parameter_vector().detach() 
            ## quadratic form around matrix estimating fisher information 
            regularizer = (t - t0).reshape([1, -1]).matmul(self.hessian_sum) 
            regularizer = regularizer.matmul((t - t0).reshape([-1, 1]))
            regularizer *= .5 / self.hessian_denominator 
            regularizer = regularizer.reshape([])
        return prediction, target, regularizer 
    
    def get_parameter_vector(self): 
        return nn.utils.parameters_to_vector(self.parameters()) 
    
    def optimize(self, max_iter=None, batch_size=None): 
        iter_n = 0 
        n_dels = 30 
        dels = [None]*n_dels 
        continue_iterating = True 
        halt_method = None 
        loss_f = None 
        mean_reward = None 
        target_model = self.copy() 
        target_model.eval() 
        while continue_iterating: 
            prev_theta = self.get_parameter_vector() 
            predicted, target, regularizer = self.__memory_replay(target_model=target_model, batch_size=batch_size) 
            mean_reward = predicted.mean() 
            #loss = F.mse_loss(predicted, target) 
            loss = F.smooth_l1_loss(predicted, target) 
            if regularizer is not None: 
                #n = predicted.shape[0] 
                #loss = (n*loss + regularizer)/(n + self.) ## TODO balance by sample size 
                loss += regularizer 
            loss_f = float(loss) 
            loss.backward() 
            if not self.lbfgs: 
                ## lbfgs really doesn't like this 
                nn.utils.clip_grad_norm_(model.parameters(), self.grad_clip) 
            ## lbfgs must re-evaluate target, hence lambda 
            if self.lbfgs: 
                self.optimizer.step(lambda: float(F.smooth_l1_loss(predicted, target))) 
            else:
                self.optimizer.step() 
                pass 
            updated_theta = self.get_parameter_vector() 
            ## decide to continue iterating or not 
            if max_iter is not None: 
                if iter_n > max_iter: 
                    ## halt: iters have hit limit 
                    continue_iterating = False 
                    halt_method = 'max-iter' 
            if iter_n >= n_dels: 
                ## test convergence with chebyshev ineq 
                dels = dels[1:] + [(updated_theta - prev_theta).abs().sum()] 
                sigma = torch.tensor(dels).square().mean().sqrt() 
                if (sigma/self.eps)**2 < .95: 
                    ## halt: convergance 
                    continue_iterating = False 
                    halt_method = 'cauchy-convergence' 
            else: 
                ## collect data for variance estimate 
                dels[iter_n] = (updated_theta - prev_theta).abs().sum() 
                pass 
            iter_n += 1 
            pass 
        return loss_f, halt_method, mean_reward  
    
    def simulate(self, fit=True, total_iters=10000, plot_rewards=True, plot_prob_func=True): 
        if plot_prob_func:
            plt.plot([self.explore_probability_func(idx) for idx in range(total_iters)]) 
            plt.show()
            pass 
        env = gym.make(self.env_name) 
        env_state = env.reset() 
        env_state_list = [torch.tensor(env_state) for _ in range(self.short_term_memory_length)] 
        env_state = torch.cat(env_state_list) 
        last_start = 0 
        last_total_reward = 0 
        n_restarts = 0 
        self.total_rewards = [] 
        ## run experiment 
        iters = tqdm(range(total_iters), disable=False) 
        for iter_idx in iters: 
            prev_env_state = env_state 
            if self.explore_probability_func(iter_idx) > np.random.uniform(): ## TODO move to get_action 
                ## explore 
                action = env.action_space.sample()
            else: 
                ## exploit 
                self.eval() 
                action = self.get_action(env_state) 
                pass 
            env_state, reward, done, info = env.step(action) 
            env_state_list = env_state_list[1:] + [torch.tensor(env_state)] 
            env_state = torch.cat(env_state_list) 
            if done: 
                reward = 0 
                pass 
            last_total_reward += reward 
            observation = env_state, reward, done, info, prev_env_state, action 
            self.store_observation(observation) 

            if iter_idx > 30 and iter_idx % 1 == 0: 
                loss, halt_method, mean_reward = self.optimize(max_iter=1, batch_size=self.batch_size) 
                loss = float(loss) 
                mean_reward = float(mean_reward) 
                param_nan = self.get_parameter_vector().isnan().sum() 
                iters.set_description(f'n_restarts: {n_restarts}, last_total_reward: {last_total_reward}, '+\
                    f'loss: {round(loss,4)}, halt: {halt_method}, mean_reward: {round(mean_reward,2)}, action: {action}') 
                pass 

            if done: 
                env_state = env.reset() 
                env_state_list = [torch.tensor(env_state) for _ in range(self.short_term_memory_length)] 
                env_state = torch.cat(env_state_list) 
                self.total_rewards.append(last_total_reward) 
                last_total_reward = 0 
                n_restarts += 1 
                pass 
            pass 
        env.close() 

        if plot_rewards: 
            plt.plot(self.total_rewards) 
            plt.show()
        pass 

In [2]:
model = Model() 
model.simulate(total_iters=10000, plot_prob_func=False) 
#model.simulate(total_iters=10, plot_prob_func=False) ## DEBUGGING

  0%|          | 0/10000 [00:00<?, ?it/s]/anaconda/envs/azureml_py36/lib/python3.6/site-packages/ipykernel_launcher.py:105: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/ipykernel_launcher.py:154: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
n_restarts: 19, last_total_reward: 2.0, loss: 3.006, halt: max-iter, mean_reward: 19.89, action: 1:   4%|▍         | 409/10000 [00:09<02:37, 60.94it/s] 

In [ ]:
## testing memory 
model.convert_observations_to_memory()
model.simulate(total_iters=10000, plot_prob_func=False) 